## BERT Based Sentiment Analysis Model Server
The model used here, was trained with the concept of transfer learning  i.e. taking huggingface transformers pretrained BERT model and further training it on a custom dataset of reviews. this yields a sentiment analysis model based on the prior knowledge of BERT. 
The model server is given a list of texts and outputs a list of labels corresponding to its prediction.
The labels express the sentiment of the writer towards the topic of the text:
0 for negative sentiment, 1 for neutral and 2 for positive.

The model file (~430 MB), can be downloaded to your local environment from: https://iguazio-sample-data.s3.amazonaws.com/models/model.pt

## mlconfig

In [1]:
%load_ext lab_black
from mlrun import mlconf
from bert_sentiment_analysis_serving import (
    BertSentimentClassifier,
    SentimentClassifierServing,
)
import os

mlconf.dbpath = mlconf.dbpath or "http://mlrun-api:8080"
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

## Examples locally
You may change model_dir to point at the path where model.pt file is saved

In [2]:
model_dir = "/User/demo_stocks/artifacts/models/bert_sentiment_analysis_model.pt"
model_server = SentimentClassifierServing("model-server", model_dir=model_dir)
model_server.load()

### example 1
Here we test a pretty straightforward example for positive sentiment.

In [3]:
output = model_server.predict(
    {
        "instances": [
            "I had a pleasure to work with such dedicated team. Looking forward to \
             cooperate with each and every one of them again."
        ]
    }
)

assert output[0] == 2

### example 2
Now we will test a couple more examples. These are arguably harder due to misleading words that express, on their own, an opposite sentiment comparing to the full text. 

In [4]:
output = model_server.predict(
    {
        "instances": [
            "This app is amazingly useless.",
            "As much as I hate to admit it, the new added feature is surprisingly user friendly.",
        ]
    }
)

assert output[0] == 0
assert output[1] == 2

## Remote activation
Create a function object with custom specification.

In [5]:
from mlrun import code_to_function, mount_v3io
import requests
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)

In [6]:
fn = code_to_function(
    name=items["name"],
    filename=items["spec"]["filename"],
    kind=items["spec"]["kind"],
    description=items["description"],
    categories=items["categories"],
    labels=items["labels"],
    image=items["spec"]["image"],
    requirements=items["spec"]["requirements"],
)

fn.spec.default_class = "SentimentClassifierServing"
fn.spec.max_replicas = 1
fn.spec.readiness_timeout = 500

fn.export("bert_sentiment_analysis_serving.yaml")

fn.add_model(
    "bert_classifier_v1",
    "/User/demo_stocks/artifacts/models/bert_sentiment_analysis_model.pt",
)

> 2021-02-18 11:33:03,886 [info] function spec saved to path: bert_sentiment_analysis_serving.yaml


In [7]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io

    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at
    # https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc

    fn.apply(mount_pvc("nfsvol", "nfsvol", "/home/joyan/data"))

In [8]:
addr = fn.deploy(project='nlp-servers', verbose=True)

> 2021-02-18 11:33:03,931 [info] Starting remote function deploy
2021-02-18 11:33:04  (info) Deploying function
{'level': 'info', 'message': 'Deploying function', 'name': 'nlp-servers-sentiment-analysis-serving', 'time': 1613647984110.763}
2021-02-18 11:33:04  (info) Building
{'level': 'info', 'message': 'Building', 'name': 'nlp-servers-sentiment-analysis-serving', 'time': 1613647984110.7979, 'versionInfo': 'Label: 1.5.16, Git commit: ae43a6a560c2bec42d7ccfdf6e8e11a1e3cc3774, OS: linux, Arch: amd64, Go version: go1.14.3'}
2021-02-18 11:33:04  (info) Staging files and preparing base images
{'level': 'info', 'message': 'Staging files and preparing base images', 'name': 'deployer', 'time': 1613647984168.4219}
2021-02-18 11:33:04  (info) Building processor image
{'imageName': 'nuclio/nlp-servers-nlp-servers-sentiment-analysis-serving-processor:latest', 'level': 'info', 'message': 'Building processor image', 'name': 'deployer', 'time': 1613647984169.1504}
2021-02-18 11:33:05  (info) Build c

RunError: cannot deploy Failed to deploy. Details:
Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]Downloading:   1%|          | 2.05k/213k [00:00<00:14, 14.4kB/s]Downloading:  25%|██▌       | 54.3k/213k [00:00<00:07, 20.3kB/s]Downloading:  58%|█████▊    | 124k/213k [00:00<00:03, 28.4kB/s] Downloading: 100%|██████████| 213k/213k [00:00<00:00, 492kB/s] 
Exception raised while running init_context [worker_id="0"]
Caught unhandled exception while initializing [traceback="Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/mlrun/serving/states.py", line 330, in init_object
    self._object = self._class_object(**class_args)
TypeError: __init__() got an unexpected keyword argument 'model_path'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/nuclio/_nuclio_wrapper.py", line 350, in run_wrapper
    args.trigger_name)
  File "/opt/nuclio/_nuclio_wrapper.py", line 80, in __init__
    

## Remote example
We will send a sentence to the model server via HTTP request. Note that the url below uses model server notation that directs our event to the predict function.

In [ ]:
import json

event_data = {'instances': ['I had a somewhat ok experience buying at that store.']}

resp = requests.put(addr + '/bert_classifier_v1/predict', json=json.dumps(event_data))

In [ ]:
print(resp.text)

The model server classified the sentence as neutral. 